In [11]:
# Summary
# The following code imports training images from the Intel Image Classification Dataset, trains a TF model, and then saves weights that can later be used on testing images.

# Intel Dataset Link
# https://www.kaggle.com/datasets/puneet6060/intel-image-classification

# Import Statements
import pandas as pd
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.utils import load_img, img_to_array
from keras.models import load_model
from keras import Sequential


Create TF Model, Train, and Compile

In [12]:
# Load Training CSV File
df = pd.read_csv('Intel_train_labels.csv')

# Preset Image Size
img_height = 150
img_width = 150
batch_size = 32


In [13]:
# Define Data Generator
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [14]:
# Load Training Images
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='Intel_seg_train',
    x_col='File Name',
    y_col='Category',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='training'
)

Found 11228 validated image filenames belonging to 6 classes.


In [15]:
# Load Testing Images
validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='Intel_seg_train',
    x_col='File Name',
    y_col='Category',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='validation'
)

Found 2806 validated image filenames belonging to 6 classes.


In [18]:
# Define TF Image Recognition Model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-5)),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Define Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile Model with Optimizer
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [20]:
# Train Model using model.fit
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10
)

# Save Model
model.save('ECE50024_ImgRec_TF_Model_V3.h5')

Epoch 1/10
374/374 [==============================] - 337s 901ms/step - loss: 0.9089 - accuracy: 0.6689 - val_loss: 10.0001 - val_accuracy: 0.1925
Epoch 2/10
374/374 [==============================] - 289s 774ms/step - loss: 0.6986 - accuracy: 0.7460 - val_loss: 12.7783 - val_accuracy: 0.1624
Epoch 3/10
374/374 [==============================] - 289s 773ms/step - loss: 0.6488 - accuracy: 0.7680 - val_loss: 14.4291 - val_accuracy: 0.1943
Epoch 4/10
374/374 [==============================] - 294s 787ms/step - loss: 0.6084 - accuracy: 0.7872 - val_loss: 12.0313 - val_accuracy: 0.2011
Epoch 5/10
374/374 [==============================] - 291s 779ms/step - loss: 0.5717 - accuracy: 0.8094 - val_loss: 13.2621 - val_accuracy: 0.1864
Epoch 6/10
374/374 [==============================] - 289s 774ms/step - loss: 0.5728 - accuracy: 0.8114 - val_loss: 13.7154 - val_accuracy: 0.2054
Epoch 7/10
374/374 [==============================] - 286s 766ms/step - loss: 0.5479 - accuracy: 0.8224 - val_loss: 15

In [22]:
# Export Trained Image Recognition Weights
model.save('TF_Models/ECE50024_ImgRec_TF_Model_V3.h5')

In [21]:
# Output Summary of CNN
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 350464)            0         
                                                                 
 dense_8 (Dense)             (None, 128)               44859520  
                                                                 
 dense_9 (Dense)             (None, 6)                 774       
                                                                 
Total params: 44,862,086
Trainable params: 44,862,086
Non-trainable params: 0
__________________________________________

Use Model to Classify Testing Image Folder's Images

In [10]:
# Predict on Folder of Testing Images

# Load Trained Model Weights
model = load_model('ECE50024_ImgRec_TF_Model_V3.h5')

# Define Path to Folder with Testing Images
folder_path = 'Intel_seg_pred'

# Initialize Results Variable
results = []

# Iterate Through Each Image in Folder
for filename in os.listdir(folder_path):
    # Load the image
    img = load_img(os.path.join(folder_path, filename), target_size=(img_height, img_width))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Make a Prediction on Images
    prediction = model.predict(img_array)

    # Extract Prediction
    category_index = np.argmax(prediction)
    category = list(train_generator.class_indices.keys())[category_index]

    # Append File Name and Predicted Category
    results.append({'filename': filename, 'category': category})

    # Print File Name and Prediction
    print('{} - {}'.format(filename, category))
    
# Convert Results to Pandas Dataframe
df = pd.DataFrame(results)

1/1 [==============================] - 0s 127ms/step
10060.jpg - forest
1/1 [==============================] - 0s 28ms/step
10082.jpg - street
1/1 [==============================] - 0s 29ms/step
10090.jpg - street
1/1 [==============================] - 0s 30ms/step
101.jpg - glacier
1/1 [==============================] - 0s 28ms/step
10106.jpg - sea
1/1 [==============================] - 0s 27ms/step
10158.jpg - glacier
1/1 [==============================] - 0s 28ms/step
10166.jpg - mountain
1/1 [==============================] - 0s 27ms/step
10173.jpg - glacier
1/1 [==============================] - 0s 27ms/step
10225.jpg - forest
1/1 [==============================] - 0s 28ms/step
10307.jpg - glacier
1/1 [==============================] - 0s 30ms/step
1036.jpg - mountain
1/1 [==============================] - 0s 28ms/step
1039.jpg - glacier
1/1 [==============================] - 0s 27ms/step
10424.jpg - forest
1/1 [==============================] - 0s 28ms/step
1046.jpg - sea
1/1 [==

In [12]:
# Save Results to CSV File
df.to_csv('img_recognition_predictions.csv', index=False)
# Exports pandas dataframe to csv file